In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from os.path import isfile, join
import errno

def convert_to_square(image):
    square_size = np.max(image.shape)
    h, w = image.shape
    delta_w = square_size - w
    delta_h = square_size - h
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    square_image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_REPLICATE)
    return square_image

def expand(image):
    if len(image.shape)==2:
        image = np.expand_dims(image, -1)
        image = np.repeat(image, 3, 2)
    return image

def get_dataset():
    x_set = []
    y_set = []
    taxons_dirs = next(os.walk(DATASET_PATH))[1]
    n_taxons = len(taxons_dirs)
    disp_progress = display("0/"+str(n_taxons),display_id=True)
    for i, taxon in enumerate(taxons_dirs):
        if not taxon in id_map:
            print("WARNING: Taxon",taxon,"not found in id_map !")
            continue
        taxon_id = id_map[taxon]
        path = join(DATASET_PATH, taxon)
        files = [f for f in os.listdir(path) if isfile(join(path, f))]
        for file in files:
            x_set.append(join(path, file))
            y_set.append(taxon_id)
        disp_progress.update(str(i+1)+"/"+str(n_taxons))
    return x_set, y_set

def test_dataset(dataset):
    for image, label in dataset.take(1):
        print("Image shape: ", image.numpy().shape)
        print("Label: ", label)
        display(Image.fromarray(np.uint8(image.numpy()[0]*255)))
        break